## Optimizing eRHIC cooler parameters 

This notebook is intended to develop the tools necessary to optimize a cooling configuration. It's also meant to be a generic testbed for developing future Sirepo features. 

Start by checking out and compiling JSPEC:

    git clone https://github.com/radiasoft/electroncooling.git
    cd electroncooling
    mkdir build
    cd build
    cmake ..
    make -j5
    cp $HOME/jupyter/StaffScratch/sjcrs/eRHIC.tfs .

This `.tfs` file was produced by madx from the eRHIC lattice supplied by Boaz. Once you have compiled JSPEC in your local directory, set the path to your local installation's 'build' path in `buildpath`

Also, if you have access to a RSMPI node, set `RSMPI=True` here

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy import optimize

buildpath = "/home/vagrant/jupyter/github/electroncooling/build/"
RSMPI=False

#### Pass the parameters of the proposed eRHIC system to JSPEC:

We create an object with the proposed eRHIC values stored, so we can simply and easily modify one of them & re-run JSPEC. We then define a function that takes this object as an argument, writes the input file, and runs the job (either locally or submitted to the RSMPI cluster). 

Open the first collapsed cell below to see all of the parameters and their default values.

The second collapsed cell below contains a class that defines the same parameters and sets the default values for the Booster example on Sirepo/JSPEC. 

The third collapsed cell contains the function `Update_eRHIC_Config(obj,rsmpi=False)` that parses this class into an input file and runs the JSPEC executable. If you have access to a RSMPI node, pass `rsmpi=True` and you will experience a ~4x speedup

In [ ]:
class JSPEC_config:
    def __init__(self):
        self.dynamic          = True #Follow through many turns or just calculate instantaneous rates?
        self.suffix           = "dump"
        self.ion_mass         = 938.272
        self.ke               = 2.5e4 # 25 GeV in MeV
        self.charge_number    = 1
        self.norm_emit_x      = 2.5e-06
        self.norm_emit_y      = 2.5e-06
        self.p_spread         = 0.001
        self.particle_number  = 1.34e12
        self.rms_bunch_length = 0.7
        self.lattice          = "eRHIC.tfs"
        self.nu               = 100
        self.nv               = 100
        self.log_c            = 20.6  #Total guess
        self.coupling         = 0.0
        self.length           = 130.0 # in meters
        self.section_number   = 1
        self.bfield           = 1.0   # in Tesla
        self.bet_x            = 10.0  #Placeholder values, take value from lattice ?
        self.bet_y            = 10.0  # ^
        self.disp_x           = 0.0   # ^
        self.disp_y           = 0.0   # ^
        self.alpha_x          = 0.0   # ^
        self.alpha_y          = 0.0   # ^
        self.disp_dx          = 0.0   # ^
        self.disp_dy          = 0.0   # ^
        self.temp_tr          = 0.01  #in eV
        self.temp_l           = 0.01  #in eV
        self.shape            = "bunched_gaussian"
        self.radius           = 0.009  # in meters
        self.current          = 4.0    # electron current in Amperes
        self.sigma_x          = 0.015  #Just a guess
        self.sigma_y          = 0.015  #^
        self.sigma_z          = 0.015  #^
        self.ebeam_length     = 0.05   # in meters
        self.e_number         = 1.5605e10
        self.sample_number    = 10000.0
        self.force_formula    = "parkhomchuk"  #Could be DERBENEVSKRINSKY, MESHKOV
        self.ibs              = "on"
        self.ecool            = "on"
        self.time             = 2000.0
        self.step_number      = 100 #so, 2000/100 = 20s samplings by default
        self.model            = "particle" #or "RMS"
        self.ref_bet_x        = 10.0
        self.ref_bet_y        = 10.0
        self.ref_alf_x        = 0.0
        self.ref_alf_y        = 0.0
        self.ref_disp_x       = 0.0
        self.ref_disp_y       = 0.0
        self.ref_disp_dx      = 0.0
        self.ref_disp_dy      = 0.0

In [ ]:
#Implement the Sirepo default Booster config for testing
class BOOSTER_config:
    def __init__(self):
        self.suffix           = "dump"
        self.ion_mass         = 938.272
        self.ke               = 2.2418e7
        self.charge_number    = 1
        self.norm_emit_x      = 2.2e-06
        self.norm_emit_y      = 2.2e-06
        self.p_spread         = 0.001
        self.particle_number  = 6.58e13
        self.rms_bunch_length = 7.0
        self.lattice          = "Booster.tfs"
        self.nu               = 100
        self.nv               = 100
        self.log_c            = 20.6
        self.coupling         = 0.0
        self.length           = 3.4
        self.section_number   = 1
        self.bfield           = 0.039
        self.bet_x            = 10.0
        self.bet_y            = 10.0
        self.disp_x           = 0.0
        self.disp_y           = 0.0
        self.alpha_x          = 0.0
        self.alpha_y          = 0.0
        self.disp_dx          = 0.0
        self.disp_dy          = 0.0
        self.temp_tr          = 0.1
        self.temp_l           = 0.01
        self.shape            = "dc_uniform"
        self.radius           = 0.004
        self.current          = 2.0
        self.sigma_x          = 0.015
        self.sigma_y          = 0.015
        self.sigma_z          = 0.015
        self.ebeam_length     = 2
        self.e_number         = 1.5605e10
        self.sample_number    = 10000.0
        self.force_formula    = "parkhomchuk"
        self.ibs              = "on"
        self.ecool            = "on"
        self.time             = 200000.0
        self.step_number      = 100
        self.model            = "particle"
        self.ref_bet_x        = 10.0
        self.ref_bet_y        = 10.0
        self.ref_alf_x        = 0.0
        self.ref_alf_y        = 0.0
        self.ref_disp_x       = 0.0
        self.ref_disp_y       = 0.0
        self.ref_disp_dx      = 0.0
        self.ref_disp_dy      = 0.0

In [ ]:
def Update_eRHIC_Config(obj,rsmpi=False):
    
    os.chdir(buildpath)
    
    file1 = open("jspec_"+obj.suffix+".in","w")#write mode     
    file1.write("\n")
    file1.write("section_scratch\n")
    file1.write("        ion_mass = "+str(obj.ion_mass)+"\n")
    file1.write("        ion_ke = "+str(obj.ke)+"\n")
    file1.write("        ion_gamma = 1 + ion_ke/ion_mass\n")
    file1.write("section_ion\n")
    file1.write("        charge_number = "+str(obj.charge_number)+"\n")
    file1.write("        mass = ion_mass\n")
    file1.write("        kinetic_energy = ion_ke\n")
    file1.write("        norm_emit_x = "+str(obj.norm_emit_x)+"\n")
    file1.write("        norm_emit_y = "+str(obj.norm_emit_y)+"\n")
    file1.write("        momentum_spread = "+str(obj.p_spread)+"\n")
    file1.write("        particle_number = "+str(obj.particle_number)+"\n")
    file1.write("        rms_bunch_length = "+str(obj.rms_bunch_length)+"\n")
    file1.write("section_ring\n")
    file1.write("        lattice = "+obj.lattice+"\n")
    file1.write("section_ibs\n")
    file1.write("        nu = "+str(obj.nu)+"\n")
    file1.write("        nv = "+str(obj.nv)+"\n")
    file1.write("        log_c = "+str(obj.log_c)+"\n")
    file1.write("        coupling = "+str(obj.coupling)+"\n")
    file1.write("section_cooler\n")
    file1.write("        length = "+str(obj.length)+"\n")
    file1.write("        section_number = "+str(obj.section_number)+"\n")
    file1.write("        magnetic_field = "+str(obj.bfield)+"\n")
    file1.write("        bet_x = "+str(obj.bet_x)+"\n")
    file1.write("        bet_y = "+str(obj.bet_y)+"\n")
    file1.write("        disp_x = "+str(obj.disp_x)+"\n")
    file1.write("        disp_y = "+str(obj.disp_y)+"\n")
    file1.write("        alpha_x = "+str(obj.alpha_x)+"\n")
    file1.write("        alpha_y = "+str(obj.alpha_y)+"\n")
    file1.write("        disp_dx = "+str(obj.disp_dx)+"\n")
    file1.write("        disp_dy = "+str(obj.disp_dy)+"\n")
    file1.write("section_e_beam\n")
    file1.write("        gamma = ion_gamma\n")
    file1.write("        tmp_tr = "+str(obj.temp_tr)+"\n")
    file1.write("        tmp_l = "+str(obj.temp_l)+"\n")
    file1.write("        shape = "+obj.shape+"\n")
    file1.write("        radius = "+str(obj.radius)+"\n")
    file1.write("        current = "+str(obj.current)+"\n")
    file1.write("        sigma_x = "+str(obj.sigma_x)+"\n")
    file1.write("        sigma_y = "+str(obj.sigma_y)+"\n")
    file1.write("        sigma_z = "+str(obj.sigma_z)+"\n")
    file1.write("        length = "+str(obj.ebeam_length)+"\n")
    file1.write("        e_number = "+str(obj.e_number)+"\n")
    file1.write("section_ecool\n")
    file1.write("        sample_number = "+str(obj.sample_number)+"\n")
    file1.write("        force_formula = "+obj.force_formula+"\n")
    file1.write("section_run\n")
    file1.write("        create_ion_beam\n")
    file1.write("        create_ring\n")
    file1.write("        create_e_beam\n")
    file1.write("        create_cooler\n")
    file1.write("section_simulation\n")
    file1.write("        ibs = "+obj.ibs+"\n")
    file1.write("        e_cool = "+obj.ecool+"\n")
    file1.write("        time = "+str(obj.time)+"\n")
    file1.write("        step_number = "+str(obj.step_number)+"\n")
    file1.write("        output_file = JSPEC"+obj.suffix+".SDDS\n")
    file1.write("        model = "+obj.model+"\n")
    file1.write("        ref_bet_x = "+str(obj.ref_bet_x)+"\n")
    file1.write("        ref_bet_y = "+str(obj.ref_bet_y)+"\n")
    file1.write("        ref_alf_x = "+str(obj.ref_alf_x)+"\n")
    file1.write("        ref_alf_y = "+str(obj.ref_alf_y)+"\n")
    file1.write("        ref_disp_x = "+str(obj.ref_disp_x)+"\n")
    file1.write("        ref_disp_y = "+str(obj.ref_disp_y)+"\n")
    file1.write("        ref_disp_dx = "+str(obj.ref_disp_dx)+"\n")
    file1.write("        ref_disp_dy = "+str(obj.ref_disp_dx)+"\n")
    file1.write("section_run\n")
    if obj.dynamic is True:
        file1.write("        run_simulation\n")
    else:
        file1.write("        calculate_ecool\n")
        file1.write("        calculate_ibs\n")
        file1.write("        total_expansion_rate\n")

    file1.close()
    
    os.chdir(buildpath)
    
    #With this option, the beam is sampled over the course of multiple turns
    if obj.dynamic is True:
        if rsmpi:
            os.system('rsmpi -n 1 ./jspec jspec_'+obj.suffix+'.in' )
        else:
            os.system('./jspec jspec_'+obj.suffix+'.in')
            
        return #Parsing output happens elsewhere for a dynamic run
    
    else:
        #Run and create a file with the instantaneous rates
        if rsmpi:
            os.system('rsmpi -n 1 ./jspec jspec_'+obj.suffix+'.in > instantaneous_rates.txt' )
        else:
            os.system('./jspec jspec_'+obj.suffix+'.in > instantaneous_rates.txt')
            
        #Parse that file and return the values
        rate_file = open(r"instantaneous_rates.txt","r")
        #Skip the first 4 lines
        [rate_file.readline() for i in range(4)]
        e_rate = rate_file.readline()
        ibs_rate = rate_file.readline()
        total_rate = rate_file.readline()
        rate_file.close()
        
        def parse(rate):
            parse = (rate.split(":")[1].rstrip()).split("  ")
            parse = [float(i) for i in parse]
            return parse
        #The 3 parameters are horizontal, vertical, longitudinal
        return parse(e_rate), parse(ibs_rate), parse(total_rate)

### Visualization Demo

Here we define a function to make a grid of plots of output parameters. We can use this function to overlay the output from one or more sets of configuration parameters. 

This is useful for making comparisons that are not yet possible in Sirepo. 

In [ ]:
def DisplayTrial(sdds_file,label=None):
    names=["t","emit_x","emit_y","dp/p","sigma_s",
           "rx","ry","rs","rx_ibs","ry_ibs","rs_ibs","rx_ecool","ry_ecool","rs_ecool",
          "rf_voltage","luminosity"]
    df = pd.read_csv(sdds_file,skiprows=21,
                     index_col=False,delim_whitespace=True,
                     names=names)
    df = df.sort_values(by='t',ascending=True)
    #Convert to microns from meters
    df["emit_x"] = df["emit_x"]*1e6
    df["emit_y"] = df["emit_y"]*1e6

   
    plt.subplot(4,3,1)
    plt.plot(df["t"],df["emit_x"],label=label)
    plt.xlabel("time")
    plt.ylabel(r"emit $x$")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.title(r"Ion $x$ Emittance")

    plt.subplot(4,3,2)
    plt.plot(df["t"],df["emit_y"])
    plt.xlabel("time")
    plt.ylabel(r"emit $y$")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.title(r"Ion $y$ Emittance")
    
    plt.subplot(4,3,3)
    plt.plot(df["t"],df["sigma_s"])
    plt.xlabel("time")
    plt.ylabel("sigma $s$")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.title("Ion Bunch Length")

#This plots the scatterplot of y vs length, with color coding time from blue to red
# showing a trajectory. Might not be useful, but I'm storing the code/idea here
#    plt.scatter(df["emit_y"],df["sigma_s"],c=df.index,cmap=plt.cm.coolwarm)
#    plt.xlabel(r"emit $y$")
#    plt.ylabel(r"sigma $s$")
#    plt.title("Time-dependent trajectory")
    
    plt.subplot(4,3,4)
    plt.plot(df["t"],df["rx"])
    plt.xlabel("time")
    plt.ylabel(r"$x$ cooling rate")
    plt.title(r"$x$ Cooling rate (Total)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.subplot(4,3,5)
    plt.plot(df["t"],df["rx_ecool"])
    plt.xlabel("time")
    plt.title(r"$x$ cooling rate")
    plt.title(r"$x$ Cooling rate (e-cool)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.subplot(4,3,6)
    plt.plot(df["t"],df["rx_ibs"])
    plt.xlabel("time")
    plt.ylabel(r"$x$ cooling rate")
    plt.title(r"$x$ Cooling rate(IBS)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.subplot(4,3,7)
    plt.plot(df["t"],df["ry"])
    plt.xlabel("time")
    plt.ylabel(r"$y$ rate")
    plt.title(r"$y$ Cooling rate (Total)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.subplot(4,3,8)
    plt.plot(df["t"],df["ry_ecool"])
    plt.xlabel("time")
    plt.ylabel(r"$y$ cooling rate")
    plt.title(r"$y$ Cooling rate (e-cool)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.subplot(4,3,9)
    plt.plot(df["t"],df["ry_ibs"])
    plt.xlabel("time")
    plt.ylabel(r"$y$ cooling rate")
    plt.title(r"$y$ Cooling rate(IBS)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    
    plt.subplot(4,3,10)
    plt.plot(df["t"],df["rs"])
    plt.xlabel("time")
    plt.ylabel(r"$s$ cooling rate")
    plt.title(r"$s$ Cooling rate (Total)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

    plt.subplot(4,3,11)
    plt.plot(df["t"],df["rs_ecool"])
    plt.xlabel("time")
    plt.ylabel(r"$s$ cooling rate")
    plt.title(r"$s$ Cooling rate (e-cool)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    
    plt.subplot(4,3,12)
    plt.plot(df["t"],df["rs_ibs"])
    plt.xlabel("time")
    plt.ylabel(r"$s$ cooling rate")
    plt.title(r"$s$ Cooling rate (IBS)")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.tight_layout()
    


In [ ]:
#Define the object with all the default values
config1 = JSPEC_config()
#Make some of our own changes
config1.bfield = 1.0
config1.model = "RMS"
config1.suffix = "DUMP1"
#config1.ibs    = "off"
#By default, this is using the Particle (macro-particle) model
A = Update_eRHIC_Config(config1,rsmpi=RSMPI)
config2 = JSPEC_config()
config2.bfield = 1.0 # User can change the solenoid field strength, in Tesla
config2.model  = "RMS"  # Switch from Particle model to using the beam moments
#config2.temp_l = 0.001 # Could vary the temperatures of the electron cooling beam
#config2.temp_tr = 0.001
#config2.ibs = "off"
config2.e_number = 3e10
config2.suffix = "DUMP2"
Update_eRHIC_Config(config2,rsmpi=RSMPI)

#Read the output files & make a standard figure
plt.clf()
plt.figure(figsize=(15,8))
cooltime1 = DisplayTrial("JSPECDUMP1.SDDS",label="4 A")
cooltime2 = DisplayTrial("JSPECDUMP2.SDDS",label="8 A")
#Only draw our legend once
plt.subplot(4,3,1)
plt.legend(loc='lower right')
plt.show()

### Parameter Scans

We can calculate the instantaneous cooling rate at $t=0$ by setting the member variable `dynamic=False`. This allows us to scan across a range of parameter values to see how the instantaneous cooling rate changes. Here we create a convencience class for parsing out the 9 possible rate values that one may want to plot.

In [ ]:
class Rates_object:
    def __init__(self,e_rate,ibs_rate,total_rate):
        #The 3 elements of each list are are horizontal, vertical, longitudinal
        self.e_rate_h = e_rate[0]
        self.e_rate_v = e_rate[1]
        self.e_rate_l = e_rate[2]
        self.ibs_rate_h = ibs_rate[0]
        self.ibs_rate_v = ibs_rate[1]
        self.ibs_rate_l = ibs_rate[2]
        self.total_rate_h = total_rate[0]
        self.total_rate_v = total_rate[1]
        self.total_rate_l = total_rate[2]

In [ ]:
#We'll draw 3 different lines, for 3 different electron transverse temperatures

plt.figure(figsize=(15,5))

for temp_tr in [0.001,0.01,0.1]:
    yval1 = []
    yval2 = []
    yval3 = []
    #The x-value will scan across different normalized ion emittances in the x direction
    xrange = np.linspace(1e-7,1e-5,50)
    for i in xrange:
    #Define the object with all the default values
        config1 = JSPEC_config()
        #Make some of our own changes
        config1.norm_emit_x = i
        config1.model = "RMS"
        config1.temp_tr = temp_tr
        config1.dynamic = False

        #Pass the function output to our parser class
        RO = Rates_object(*Update_eRHIC_Config(config1,rsmpi=RSMPI))
    
        yval1.append(RO.e_rate_h)
        yval2.append(RO.e_rate_v)
        yval3.append(RO.e_rate_l)
    
    plt.subplot(1,3,1)    
    plt.plot(xrange,yval1,label="T = "+str(temp_tr) + " eV")
    plt.xlabel("Initial normalized x emittance")
    plt.ylabel("Cooling Rate")
    plt.title("Horizontal Rate")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.subplot(1,3,2)
    plt.plot(xrange,yval2)
    plt.xlabel("Initial normalized x emittance")
    plt.ylabel("Cooling Rate")
    plt.title("Vertical Rate")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.subplot(1,3,3)
    plt.plot(xrange,yval3)
    plt.xlabel("Normalized x emittance")
    plt.title("Longitudinal Rate")

    plt.ylabel("Cooling Rate")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

plt.tight_layout()
plt.subplot(1,3,1)
plt.legend(loc='lower right')
plt.show()


In [ ]:
#Uncomment these lines below to test with the Sirepo default "booster" config
# (and also change the arguments in DisplayTrial below to match)
#config3 = BOOSTER_config()
#config3.bfield = 0.039
#config3.suffix = "DUMP3"
#Update_eRHIC_Config(config3)

#config4 = BOOSTER_config()
#config4.bfield = 0.05
#config4.suffix = "DUMP4"
#Update_eRHIC_Config(config4)

#Read the output files & make a standard figure
#plt.clf()
#plt.figure(figsize=(15,8))
#cooltime1 = DisplayTrial("JSPECDUMP3.SDDS",label="B = 0.039 T")
#cooltime2 = DisplayTrial("JSPECDUMP4.SDDS",label="B = 0.05 T")
#plt.subplot(4,3,1)
#plt.legend(loc='lower right')
#plt.show()

### Nonlinear optimization (Not Tested)

Here we could let Scipy figure out the combination of parameters that yields a 20 minute cooling time. 

In [ ]:
#For the moment, the parameters you can optimize are hard-coded in the 
# defintion of CalculateRate 
def CalculateRate(*args):
    
    #Keep an eye on the parameter search progress/attempts
    print(args)
    
    #Submit this iteration as a job & produce an output file
    config = JSPEC_config()

    config.bfield = args[0][0]
    config.p_spread = args[0][1]
    config.temp_l = args[0][2]
    config.model = "RMS" #Using moments smooths the space
    config.suffix = "_MIN" #Define the filename for checking the output
    Update_eRHIC_Config(config,rsmpi=RSMPI)   

    #Read the output and calculate the cooling time to satisfy the optimizer
    names=["t","emit_x","emit_y","dp/p","sigma_s",
           "rx","ry","rs","rx_ibs","ry_ibs","rx_ecool","ry_ecool","rs_ecool",
          "rf_voltage","luminosity"]
    df = pd.read_csv("JSPEC_MIN.SDDS",skiprows=21,
                     index_col=False,delim_whitespace=True,
                     names=names)
    df = df.sort_values(by='t',ascending=True)
    df["emit_x"] = df["emit_x"]*1e6 #Convert to um
    df["emit_y"] = df["emit_y"]*1e6 #Convert to um
    df["ry_ibs"] = (df["ry_ibs"]*1e13)
    
    #Calculate the cooling time from the rate at the last time 
    # This is a hack of an optimization metric, not intended to 
    # show anything real, just to make the function work
    cooltime =  (1./df.iloc[-1]["rx"])/60. #in minutes
        
    #Keep an eye on the parameter search progress/attempts
    print(args, cooltime)
    
    return abs(20.-cooltime)

#Key to these arguments: [bfield,p_spread,temp_l]
lb = (0.001,0.0001,0.001) #The lower bounds for each argument
ub = (10.,0.01,0.1)       #The upper bounds for each argument

#This bounds object is passed to the modern optimize.minimize function
# which seems to be newer than the scipy version on radiasoft.jupyter
bounds = optimize.Bounds(lb,ub)

bounds = [a for a in zip(lb,ub)]

# Pick a method that allows you to specify bounds
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_tnc.html
minimum = optimize.fmin_tnc(CalculateRate, 
#minimum = optimize.fmin_slsqp(CalculateRate,
                            np.array([1.5,0.001,0.01]), #Best guess starting point
                            approx_grad=True, #Since we haven't defined a jacobian, only for fmin_tnc
                            bounds=bounds,
                            disp=5) #lots of text output
    
print(minimum) #Array of [solution, #iterations,return code (see link above)]